In [2]:
import rasterio
import os
import numpy
import fiona
from shapely import geometry
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show, show_hist
from rasterio import features
import matplotlib
%matplotlib inline

In [ ]:
BEACH_AERIAL_PATH = "/Users/ciaran/data/imagery/cco/seaton/cco_data-20190107184417/data/aerial"
BEACH_GENERATED_GEOTIFF_PATH_WITHOUT_EXT = "/Users/ciaran/data/imagery/tifs/seaton/seaton_2012"
BEACH_GENERATED_GEOTIFF_PATH = BEACH_GENERATED_GEOTIFF_PATH_WITHOUT_EXT + ".tif"
BEACH_CELL_PATH_WITHOUT_EXT = "/Users/ciaran/data/imagery/tifs/seaton/train/images/seaton_"
BEACH_CELL_LABELS_PATH_WITHOUT_EXT = "/Users/ciaran/data/imagery/tifs/blue_anchor/train/labels/blue_anchor_"
BEACH_ALL_CELLS_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/cells/*"
NO_DATA_SHAPEFILE_PATH = "/Users/ciaran/data/shapefiles/blue_anchor/no_data.shp"
ROCK_SHAPEFILE_PATH = "/Users/ciaran/data/shapefiles/blue_anchor/rock.shp"
SAND_SHAPEFILE_PATH = "/Users/ciaran/data/shapefiles/blue_anchor/sand.shp"
PEBBLE_SHAPEFILE_PATH = "/Users/ciaran/data/shapefiles/blue_anchor/pebble.shp"
PEBBLE_2_SHAPEFILE_PATH = "/Users/ciaran/data/shapefiles/blue_anchor/pebble_2.shp"
RASTERISED_ROCK_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/rock.tif"
RASTERISED_SAND_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/sand.tif"
RASTERISED_PEBBLE_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/pebble.tif"
RASTERISED_PEBBLE_2_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/pebble_2.tif"
RASTERISED_NO_DATA_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/no_data.tif"
MERGED_LABELS_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/merged_labels.tif"
BLUE_ANCHOR_LABELLED_PATH = "/Users/ciaran/data/imagery/tifs/blue_anchor/blue_anchor_2013_labels.tif"
NO_DATA_VALUE=115
ROCK_VALUE = 100
SAND_VALUE = 105
PEBBLE_VALUE = 110

# Generating one big GeoTIFF from multiple aerial images of a beach

In [2]:
def generateBeachFromCells(pathToBeachImages, nameOfBeach):
    imagesToMerge = getAllImagesToMerge(pathToBeachImages)
    mergedImage, mergedImageTransform = merge(imagesToMerge)
    metadata = imagesToMerge[0].meta.copy()
    crs = imagesToMerge[0].crs
    writeImageAsGeoTIFF(mergedImage, mergedImageTransform, metadata, crs, nameOfBeach)

def getAllImagesToMerge(pathToBeachImages):
    return [rasterio.open(os.path.join(pathToBeachImages, image)) for image in os.listdir(pathToBeachImages)]

def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": crs})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [ ]:
generateBeachFromCells(BEACH_AERIAL_PATH, BEACH_GENERATED_GEOTIFF_PATH_WITHOUT_EXT)

# Open beach GeoTIFF in QGIS

In [ ]:
os.system("open -a qgis " + BEACH_GENERATED_GEOTIFF_PATH)

# Generate training data cells of squareDim * squareDim size

In [29]:
def splitImageIntoCells(img, filename, squareDim):
    numberOfCellsWide = img.shape[1] // squareDim
    numberOfCellsHigh = img.shape[0] // squareDim
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim
        for wc in range(numberOfCellsWide):
            x = wc * squareDim
            geom = getTileGeom(img.transform, x, y, squareDim)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1
            
def getTileGeom(transform, x, y, squareDim):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim, y + squareDim) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])
    
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+str(count))

In [ ]:
blue_anchor = rasterio.open(BEACH_GENERATED_GEOTIFF_PATH)

In [ ]:
blue_anchor_labelled = rasterio.open(BLUE_ANCHOR_LABELLED_PATH)

In [30]:
sand_raster = rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/only_sand.tif")

In [31]:
splitImageIntoCells(sand_raster, "/Users/ciaran/data/imagery/tifs/blue_anchor/train/labels/sand/blue_anchor_", 256)

# Open all cells in QGIS (CAREFUL - This could be 1000s!)

In [ ]:
os.system("open -a qgis " + BEACH_ALL_CELLS_PATH)

# Rasterise shapefile for training labels

In [ ]:
def getRasterisedShapefile(shapefilePath, baseCRS, baseMeta, baseShape, baseTransform, value):
    shapefile = fiona.open(shapefilePath)
    geometries = [shape['geometry'] for shape in shapefile if shape['geometry'] is not None]
    rasterisedShp = features.rasterize(geometries, out_shape=baseShape, transform=baseTransform, default_value=value)
    baseMeta.update({"driver":"GTiff",
                     "height":baseShape[0],
                     "width":baseShape[1],
                     "transform": baseTransform,
                     "crs": baseCRS,
                     "count": 1,
                     "nodata": 0})
    return (rasterisedShp, baseMeta)
    
def writeRasterisedShapefile(rasterisedShp, meta, filename):
    with rasterio.open(filename, "w", **meta) as dest:
        dest.write(rasterisedShp, 1)
        
def rasteriseShapefileAndWriteToGeoTIFF(shapefileToRasterisePath, crs, meta, shape, transform, value, outPath):
    rasterisedShp, rasterisedShpMeta = getRasterisedShapefile(shapefileToRasterisePath,
                                                              crs, 
                                                              meta,
                                                              shape,
                                                              transform,
                                                              value)
    writeRasterisedShapefile(rasterisedShp,
                             rasterisedShpMeta,
                             outPath)

In [ ]:
rasteriseShapefileAndWriteToGeoTIFF(NO_DATA_SHAPEFILE_PATH,
                                    blue_anchor.crs,
                                    blue_anchor.meta,
                                    blue_anchor.shape,
                                    blue_anchor.transform,
                                    NO_DATA_VALUE,
                                    RASTERISED_NO_DATA_PATH)

In [ ]:
rasteriseShapefileAndWriteToGeoTIFF(ROCK_SHAPEFILE_PATH,
                                    blue_anchor.crs,
                                    blue_anchor.meta,
                                    blue_anchor.shape,
                                    blue_anchor.transform,
                                    ROCK_VALUE,
                                    RASTERISED_ROCK_PATH)

In [ ]:
rasteriseShapefileAndWriteToGeoTIFF(PEBBLE_SHAPEFILE_PATH,
                                    blue_anchor.crs,
                                    blue_anchor.meta,
                                    blue_anchor.shape,
                                    blue_anchor.transform,
                                    PEBBLE_VALUE,
                                    RASTERISED_PEBBLE_PATH)
rasteriseShapefileAndWriteToGeoTIFF(PEBBLE_2_SHAPEFILE_PATH,
                                    blue_anchor.crs,
                                    blue_anchor.meta,
                                    blue_anchor.shape,
                                    blue_anchor.transform,
                                    PEBBLE_VALUE,
                                    RASTERISED_PEBBLE_2_PATH)

In [ ]:
rasteriseShapefileAndWriteToGeoTIFF(SAND_SHAPEFILE_PATH,
                                    blue_anchor.crs,
                                    blue_anchor.meta,
                                    blue_anchor.shape,
                                    blue_anchor.transform,
                                    SAND_VALUE,
                                    RASTERISED_SAND_PATH)

# Open training labels in QGIS

In [ ]:
os.system("open -a qgis " + RASTERISED_ROCK_PATH)
os.system("open -a qgis " + RASTERISED_SAND_PATH)
#os.system("open -a qgis " + RASTERISED_PEBBLE_PATH)

# Merge labels into one raster

In [ ]:
baseMeta = blue_anchor.meta
baseShape = blue_anchor.shape
baseCRS = blue_anchor.crs
baseTransform = blue_anchor.transform
baseMeta.update({"driver":"GTiff",
                 "height":baseShape[0],
                 "width":baseShape[1],
                 "transform": baseTransform,
                 "crs": baseCRS,
                 "count": 1,
                 "nodata": 0})

rock = rasterio.open(RASTERISED_ROCK_PATH)
sand = rasterio.open(RASTERISED_SAND_PATH)
pebble = rasterio.open(RASTERISED_PEBBLE_PATH)
pebble_2 = rasterio.open(RASTERISED_PEBBLE_2_PATH)
no_data = rasterio.open(RASTERISED_NO_DATA_PATH)

merged_labels, _ = merge([pebble, sand, pebble_2], nodata=0)
#out, trans = merge((rock, sand, pebble), nodata=0)

In [ ]:
with rasterio.open(MERGED_LABELS_PATH, "w", **baseMeta) as dest:
        dest.write(merged_labels)

In [ ]:
merged_without_mask_or_rock = rasterio.open(MERGED_LABELS_PATH)
shapefile = fiona.open(NO_DATA_SHAPEFILE_PATH)
geometries = [shape['geometry'] for shape in shapefile if shape['geometry'] is not None]
masked, _ = mask(merged_without_mask_or_rock, geometries, all_touched=True, invert=True)

In [ ]:
with rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/without_rock.tif", "w", **baseMeta) as dest:
        dest.write(masked)

In [ ]:
without_rock = rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/without_rock.tif")
with_rock_array, _ = merge([rock, without_rock], nodata=0)
with rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/with_rock.tif", "w", **baseMeta) as dest:
        dest.write(with_rock_array)

# Open merged labels in QGIS

In [ ]:
os.system("open -a qgis " + MERGED_LABELS_PATH)

# Extract Binary Sand || !Sand

In [ ]:
labelled_raster = rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/blue_anchor_2013_labels.tif")
not_sand_mask = labelled_raster.read() == 105
not_sand_mask = not_sand_mask.astype(numpy.uint8)

In [ ]:
baseMeta = labelled_raster.meta
baseShape = labelled_raster.shape
baseCRS = labelled_raster.crs
baseTransform = labelled_raster.transform
baseMeta.update({"driver":"GTiff",
                 "height":baseShape[0],
                 "width":baseShape[1],
                 "transform": baseTransform,
                 "crs": baseCRS,
                 "count": 1,
                 "nodata": 0})
with rasterio.open("/Users/ciaran/data/imagery/tifs/blue_anchor/only_sand.tif", "w", **baseMeta) as dest:
        dest.write(not_sand_mask)

# Normalise

In [3]:
for file in os.listdir("/Users/ciaran/data/imagery/tifs/blue_anchor/train/images"):
            if file.endswith(".tif"):
                image = rasterio.open(os.path.join("/Users/ciaran/data/imagery/tifs/blue_anchor/train/images", file))
                normalized_image = image.read()/255
                baseMeta = image.meta
                baseShape = image.shape
                baseCRS = image.crs
                baseTransform = image.transform
                baseMeta.update({"driver":"GTiff",
                                 "height":baseShape[0],
                                 "width":baseShape[1],
                                 "transform": baseTransform,
                                 "crs": baseCRS,
                                 "dtype": 'float64'})
                with rasterio.open(os.path.join("/Users/ciaran/data/imagery/tifs/blue_anchor/train/normalised_images", file), "w", **baseMeta) as dest:
                    dest.write(normalized_image)
                image = None
                normalized_image = None